Motivation of this series:
      You want to feed the cell_state (not the hidden state) of the last cell in the previous episode to 
      the first cell in the next episode

The num of time steps is determined by the input. Here time_step is one. So you are trying to feed the
the cell state to the cell in next episode


Trap: 
    you tried to update self.rnn_state_t in the following way. But it does not work.
    output_, self.rnn_state_t = self.rnn_cell(input_t, self.rnn_state_t)

You tested whether the cell state is passed to the next episode by testing the output of the same input
are equal or not in two episode. It turned out it is always the same.

In [ ]:
self.rnn_state_t = self.rnn_cell.zero_state(self.batch_size,tf.float32)
The above line set the initial state to be zeros. It works. The interface of rnn_cell should have a 
function like this which allows you to set the cell state with a tensor in one line

In [1]:
import tensorflow as tf
import numpy as np
state_size = 4
input_len =5
batch_size=3


In [2]:
class Agent:
    def __init__(self):
        self.rnn_cell = None
        self.initialized = False
        self.batch_size = None
        self.output = None
        self.rnn_state_t=None
    def first(self,input_t):
        self.batch_size = tf.unpack(tf.shape(input_t))[0]
        with tf.variable_scope("rnn"):
            self.rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(state_size)
            self.rnn_state_t = self.rnn_cell.zero_state(self.batch_size,tf.float32)
        output_= self.predict(input_t)
        self.initialized = True
        return output_
    def predict(self,input_t):

        if self.initialized:
            tf.get_variable_scope().reuse_variables()
        with tf.variable_scope("rnn/RNN"):
            output_, self.rnn_state_t = self.rnn_cell(input_t, self.rnn_state_t)

            
        return output_ , self.rnn_state_t

In [3]:
input_t = tf.placeholder(tf.float32,(None,input_len))
agent = Agent()


output1_t = agent.first(input_t)

output2_t = agent.predict(input_t)

init_op = tf.global_variables_initializer()

saver = tf.train.Saver()

In [4]:
sess = tf.Session()
sess.run(init_op)

np.random.seed(0)
input = np.random.rand(batch_size,input_len)

output1_v1,output2_v1 = sess.run([output1_t, output2_t],{input_t: input})

save_path = saver.save(sess, './rnn_checkpoints/model.ckpt')

In [5]:
output1_v2,output2_v2 = sess.run([output1_t, output2_t],{input_t: input})

In [6]:
output2_v1[0] == output2_v2[0]

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]], dtype=bool)

In [7]:
# Define a testing agent
saver.restore(sess, './rnn_checkpoints/model.ckpt')
agent_test = Agent()


output1_t = agent_test.first(input_t)
output2_t = agent_test.predict(input_t)

output1_v3,output2_v3 = sess.run([output1_t, output2_t],{input_t: input})

In [8]:
output2_v2[0] == output2_v3[0]

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]], dtype=bool)